# Numba

[Numba](https://numba.pydata.org/) 是一个开源的 JIT 编译器，能够将 Python 和 NumPy 代码的子集转换为快速的机器码。

Numba 通过直接将受限的 Python 代码子集编译为 CUDA 内核和设备函数来支持 CUDA GPU 编程，遵循 CUDA 执行模型。用 Numba 编写的内核似乎可以直接访问 NumPy 数组。NumPy 数组会在 CPU 和 GPU 之间自动传输。

## 什么是内核？

内核类似于函数，它是一个代码块，接受一些输入并由处理器执行。

函数和内核之间的区别是：
- 内核不能返回任何内容，必须修改内存
- 内核必须指定其线程层次结构（线程和块）

## 什么是网格、线程和块（以及线程束）？

[线程和块](https://en.wikipedia.org/wiki/Thread_block_(CUDA_programming)) 是你指示 GPU 并行处理某些代码的方式。我们的 GPU 是一个并行处理器，所以我们需要指定希望内核执行的次数。

线程的好处是它们之间有一些共享的缓存内存，但每个 GPU 上的核心数量有限，所以我们需要将工作分解成块，这些块将在 GPU 上并行调度和运行。

<figure>

![CPU GPU 对比](images/threads-blocks-warps.png)

<figcaption style="text-align: center;"> 
    
图片来源 <a href="https://docs.nvidia.com/cuda/cuda-c-programming-guide/">https://docs.nvidia.com/cuda/cuda-c-programming-guide/</a>
    
</figcaption>
</figure>


### 什么？？

现在不要太担心这个。只需要记住**我们需要指定希望内核被调用的次数**，这是通过两个数字相乘来给出整体网格大小的。

选择每个块的线程数的经验法则：
- 应该是线程束大小（32）的倍数
- 一个好的起点是每个块 128-512 个线程，但需要通过基准测试来确定最佳值。


## Hello world

让我们通过一些代码深入了解，希望事情会变得更清晰。

首先，让我们编写一个简单的基于 CPU 的 Python 函数，我们将在[列表推导式](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions)中重复调用它。从 Python 的角度来看，列表推导式可以是并行计算的一个很好的起点，因为它们本身已经感觉有些并行性。

In [1]:
data = range(10)

def foo(i):
    return i
    
[foo(i) for i in data]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

这里我们的 `foo` 函数返回其索引值，并使用 `for` 循环遍历由 `range` 生成的数据。

接下来，我们将把它转换为 CUDA 内核，并使用 Numba CUDA 在我们的 GPU 上运行它。

首先我们需要记住，我们的内核不能返回任何内容。相反，我们将使用一个输出列表来存储我们想要返回的值。

In [2]:
data = range(10)
output = []

def foo(i):
    output.append(i)
    
[foo(i) for i in data]

output

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

我们的下一个挑战是 GPU 上的输出数组必须具有固定长度。我们不能从一个空数组开始并不断追加内容。所以让我们使用 NumPy 创建一个与输入数据长度相同的数组。我们还将输入列表转换为 NumPy 数组，因为这是我们可以移动到 GPU 的内容。

In [4]:
import numpy as np

In [5]:
data = np.asarray(range(10))
output = np.zeros(len(data))

def foo(i):
    output[i] = i
    
[foo(i) for i in data]

output

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

现在我们的纯 Python 函数行为类似于内核，让我们使用 Numba 将其转换为真正的内核。

In [6]:
import os
os.environ['NUMBA_CUDA_USE_NVIDIA_BINDING'] = '1'

In [7]:
from numba import cuda
from numba import config as numba_config
numba_config.CUDA_ENABLE_PYNVJITLINK = True

In [8]:
data = np.asarray(range(10))
output = np.zeros(len(data))

@cuda.jit
def foo(input_array, output_array):
    i = cuda.grid(1)
    output_array[i] = input_array[i]
    
foo[1, len(data)](data, output)

output

/home/ubuntu/miniconda3/envs/acccom/lib/python3.12/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


LinkerError: [222] Call to cuLinkAddData results in CUDA_ERROR_UNSUPPORTED_PTX_VERSION
ptxas application ptx input, line 9; fatal   : Unsupported .version 8.4; current version is '8.2'

**太棒了，上面的代码在我们的 GPU 上运行了！**

现在让我们来解析一下。

要将我们的 CPU 函数转换为 GPU 内核，我们需要添加 `@cuda.jit` 装饰器。这告诉 Numba 在运行时将我们的代码编译为 CUDA 兼容的字节码。

接下来，我们将内核的输入更改为 `input_array` 和 `output_array`。这是因为我们的内核需要引用这两个数组才能与它们交互。（稍后会详细介绍。）

但是 `i` 呢？我们不再需要在每次调用函数时传递索引，而是可以依赖一个很好的 CUDA 函数 `cuda.grid`，它允许我们的内核在运行时获取自己的线程索引。

最后我们做了一个看起来奇怪的函数调用 `foo[blocks, threads](input, output)`。为了在 GPU 上并行运行我们的内核，我们需要指定希望它运行的次数。内核函数使用方括号配置，传递块大小和线程大小。由于我们的数组只有 `10` 个元素长，我们指定块大小为 `1`，线程大小为 `10`，这意味着我们的内核将被执行 `10` 次。然后我们像往常一样传递参数。

## 稍微大一点的东西

现在我们已经用 Numba 运行了第一个 CUDA 内核，让我们尝试一些稍微大一点的东西。

这次我们将取一个大型数组，并将其中的每个数字加倍。我们首先在 CPU 上用纯 Python 实现，然后在 GPU 上用 CUDA 内核实现。

让我们从一个包含 3000 万个随机数的大型数组开始，以及一个等长的输出数组。

In [ ]:
random_array = np.random.random((30_000_000))
random_array

In [ ]:
output = np.zeros_like(random_array)
output

然后在 Python 中，让我们遍历这个数组，并将每个项加倍到输出数组中。我们可以计时这个单元格，看看需要多长时间。

In [ ]:
%%time

def foo(i):
    output[i] = random_array[i] * 2
    
[foo(i) for i in range(len(random_array))]

output

对我来说，CPU 完成这个计算大约需要 10 秒。

接下来，让我们编写一个 CUDA 内核，做完全相同的事情。与上一个示例的唯一区别是，我们将线程大小设置为固定值 `128`，然后计算需要多少个块来覆盖整个数组。

In [ ]:
import math

In [ ]:
%%time

output = np.zeros_like(random_array)

threads = 128
blocks = math.ceil(random_array.shape[0] / threads)

@cuda.jit
def foo(input_array, output_array):
    i = cuda.grid(1)
    output_array[i] = input_array[i] * 2
    
foo[blocks, threads](random_array, output)
output

太棒了！现在速度快了几个数量级，只需要几百毫秒。

精明的你可能会想，NumPy 已经是一个基于 C 的优化库，我们正在将 GPU 内核与一些纯 Python 代码进行比较。如果我们用 NumPy 实现会怎样？

嗯，你是对的，在这个例子中，NumPy 仍然比我们的 GPU 快。

In [ ]:
%%time 

random_array * 2

但这种情况的原因是内存管理。

## 内存管理

之前我们讨论过，CPU 和 GPU 实际上是两台独立的计算机。每台计算机都有自己的内存。

到目前为止我们处理的所有数据都是在 CPU 上用 `numpy` 创建的。因此，为了让 `numba` 在 GPU 上处理这些数据，它一直在悄悄地为我们来回复制数据。

这种数据移动会带来性能损失。

我们也可以选择自己控制数据。我们可以使用 `cuda.to_device` 提前将数组显式移动到 GPU。

In [ ]:
gpu_random_array = cuda.to_device(random_array)
gpu_output = cuda.to_device(np.zeros_like(random_array))

In [ ]:
gpu_random_array

现在如果我们再次运行内核，并将 GPU 内存数组传递给它，我们会看到它确实比 NumPy 表现更好。

In [ ]:
%%timeit -n 100
foo[blocks, threads](gpu_random_array, gpu_output)
cuda.synchronize()  # 等待内核计算，因为它会延迟完成

然而我们的输出结果也仍然在 GPU 上。我们显式地将其复制到了那里，所以我们需要使用 `copy_to_host()` 显式地将其复制回来。

In [ ]:
gpu_output.copy_to_host()

这两种数据移动操作都需要时间。但我们在这里执行的计算是微不足道的。随着内核内的计算变得越来越复杂，复制数据所花费的时间比例会变得越来越小。

内存管理在其他地方也很有用。我们可能希望编写代码，在其中编写许多内核并将它们链接在一起。在每个内核调用之间将数据复制到 GPU 并再次复制回来将是低效的。

```python
# 将数组移动到 GPU
foo[blocks, threads](data, output)
# 将数据移回 CPU

# 将数组移动到 GPU
bar[blocks, threads](data, output)
# 将数据移回 CPU

# 将数组移动到 GPU
baz[blocks, threads](data, output)
# 将数据移回 CPU
```

因此，通过显式地将数据放在那里，我们可以减少这个时间，并更好地控制我们的计算。

```python
# 将数组移动到 GPU
data = cuda.to_device(data)
output = cuda.to_device(output)

foo[blocks, threads](data, output)
bar[blocks, threads](data, output)
baz[blocks, threads](data, output)

# 将数据移回 CPU
data = data.copy_to_host()
output = output.copy_to_host()
```